In [2]:
%pip install nbformat

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import plotly.express as px
from ipywidgets import ToggleButtons, interact
from datetime import datetime

# Load data
df = pd.read_excel(r'C:\Users\TUF\Desktop\Franz FIles\Balaji-Fast-Food-Sales-Cleaned.xlsx')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.dropna(subset=['date', 'transaction_amount'], inplace=True)
df.rename(columns={
    'transaction_amount': 'Revenue',
    'item_type': 'Category'
}, inplace=True)

# Create month and quarter
df['Month'] = df['date'].dt.to_period('M').dt.to_timestamp()
df['Quarter'] = df['date'].dt.to_period('Q').dt.to_timestamp()

# Use colors that pop on dark backgrounds
custom_colors = ['#ffd166', '#06d6a0', '#ef476f', '#118ab2', '#ffffff']

# MONTHLY
summary = df.groupby('Month')['Revenue'].sum().reset_index()
fig = px.line(
    summary, x='Month', y='Revenue',
    title='Monthly Sales Trend',
    markers=True,
    color_discrete_sequence=custom_colors
)
fig.update_layout(
    template='plotly_white',
    height=500,
    xaxis_title='Date',
    yaxis_title='Revenue',
    title_font=dict(size=22, family='Arial', color=custom_colors[2]),
    font=dict(family='Arial', color='#fff'),
    plot_bgcolor='#3a4f63',
    paper_bgcolor='#3a4f63'
)
fig.write_html("monthly_sales_chart_monthly.html")
fig.show()
# QUARTERLY
summary = df.groupby('Quarter')['Revenue'].sum().reset_index()
fig = px.line(
    summary, x='Quarter', y='Revenue',
    title='Quarterly Sales Trend',
    markers=True,
    color_discrete_sequence=custom_colors
)
fig.update_layout(
    template='plotly_white',
    height=500,
    xaxis_title='Date',
    yaxis_title='Revenue',
    title_font=dict(size=22, family='Arial', color=custom_colors[2]),
    font=dict(family='Arial', color='#fff'),
    plot_bgcolor='#3a4f63',
    paper_bgcolor='#3a4f63'
)
fig.write_html("monthly_sales_chart_quarterly.html")

# BY TYPE
summary = df.groupby(['Month', 'Category'])['Revenue'].sum().reset_index()
fig = px.line(
    summary, x='Month', y='Revenue', color='Category',
    title='Monthly Sales by Item Type',
    markers=True,
    color_discrete_sequence=custom_colors
)
fig.update_layout(
    template='plotly_white',
    height=500,
    xaxis_title='Date',
    yaxis_title='Revenue',
    title_font=dict(size=22, family='Arial', color=custom_colors[2]),
    font=dict(family='Arial', color='#fff'),
    plot_bgcolor='#3a4f63',
    paper_bgcolor='#3a4f63'
)
fig.write_html("monthly_sales_chart_bytype.html")

# GROWTH
monthly = df.groupby('Month')['Revenue'].sum().reset_index()
monthly['Growth'] = monthly['Revenue'].pct_change() * 100
fig = px.bar(
    monthly, x='Month', y='Growth',
    title='Month-over-Month Growth (%)',
    color_discrete_sequence=custom_colors
)
fig.update_layout(
    template='plotly_white',
    height=500,
    xaxis_title='Date',
    yaxis_title='Growth %',
    title_font=dict(size=22, family='Arial', color=custom_colors[2]),
    font=dict(family='Arial', color='#fff'),
    plot_bgcolor='#3a4f63',
    paper_bgcolor='#3a4f63'
)
fig.write_html("monthly_sales_chart_growth.html")
fig.show()

In [4]:
# Import necessary libraries
import pandas as pd
import plotly.express as px
import os

# Load and clean the data
df = pd.read_excel(r'C:\Users\TUF\Desktop\Franz FIles\Balaji-Fast-Food-Sales-Cleaned.xlsx')
df = df.dropna(subset=['item_name', 'quantity', 'transaction_amount', 'item_type'])

# Rename columns for clarity
df.rename(columns={
    'item_name': 'Item',
    'item_type': 'Category',
    'transaction_amount': 'Revenue'
}, inplace=True)

# Aggregate data per item
summary = df.groupby(['Item', 'Category']).agg({
    'quantity': 'sum',
    'Revenue': 'sum'
}).reset_index()

# Compute average price per item
summary['Avg_Price'] = summary['Revenue'] / summary['quantity']

# Custom color palette for dark background
custom_colors = ['#ffd166', '#06d6a0', '#ef476f', '#118ab2', '#ffffff']

# Prepare output directory
os.makedirs("item_scatter_charts", exist_ok=True)

# Get unique categories including ALL
categories = ['ALL'] + sorted(df['Category'].dropna().unique())

# Generate scatter chart for each category
for cat in categories:
    if cat == 'ALL':
        filtered = summary
    else:
        filtered = summary[summary['Category'].str.lower() == cat.lower()]  # FIXED

    fig = px.scatter(
        filtered,
        x='quantity',
        y='Revenue',
        color='Category',
        hover_data=['Item', 'Avg_Price'],
        text='Item',
        title=f"Sales Conversion vs Quantity ({cat})",
        size='Avg_Price',
        template='plotly_white',
        color_discrete_sequence=custom_colors
    )

    fig.update_traces(
        textposition='top center',
        marker=dict(line=dict(width=1, color='DarkSlateGrey'))
    )

    fig.update_layout(
        xaxis_title='Total Quantity Sold',
        yaxis_title='Total Revenue (₱)',
        height=550,
        font=dict(family='Arial', color='#fff'),
        plot_bgcolor='#3a4f63',
        paper_bgcolor='#3a4f63',
        title_font=dict(size=22, family='Arial', color=custom_colors[2]),
    )

    # Save HTML chart
    fname = f"item_scatter_charts/item_scatter_{cat.replace(' ', '_').lower()}.html"
    fig.write_html(fname)
    fig.show()


In [5]:
import pandas as pd
import plotly.express as px

# Load data
df = pd.read_excel(r'C:\Users\TUF\Desktop\Franz FIles\Balaji-Fast-Food-Sales-Cleaned.xlsx')

# Clean and prepare
df['transaction_amount'] = pd.to_numeric(df['transaction_amount'], errors='coerce')
df = df.dropna(subset=['transaction_amount'])
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Group and plot
sales_by_time = df.groupby(['date', 'time_of_sale'])['transaction_amount'].sum().reset_index()
fig_line = px.line(
    sales_by_time,
    x='date',
    y='transaction_amount',
    color='time_of_sale',
    title='Sales Performance by Time of Sale Period',
    labels={'date': 'Date', 'transaction_amount': 'Total Sales', 'time_of_sale': 'Time of Sale'}
)

# Apply custom background color and white text
fig_line.update_layout(
    plot_bgcolor='#3a4f63',
    paper_bgcolor='#3a4f63',
    font=dict(color='white')
)

# Show and export
fig_line.show()
fig_line.write_html("salesperformance.html")


In [6]:
import pandas as pd
import plotly.express as px

# Load and clean
df = pd.read_excel(r'C:\Users\TUF\Desktop\Franz FIles\Balaji-Fast-Food-Sales-Cleaned.xlsx')
df['transaction_amount'] = pd.to_numeric(df['transaction_amount'], errors='coerce')
df = df.dropna(subset=['transaction_amount'])

# Compute metrics
item_metrics = df.groupby('item_name').agg(
    Revenue=('transaction_amount', 'sum'),
    Quantity=('quantity', 'sum'),
    Frequency=('order_id', 'count'),
    AvgPrice=('item_price', 'mean')
).reset_index()

# --- Revenue Chart ---
chart_data_revenue = item_metrics.sort_values(by='Revenue', ascending=False).head(10)
fig_revenue = px.bar(
    chart_data_revenue,
    x='item_name',
    y='Revenue',
    title='Top Items by Revenue',
    color='item_name',
    text_auto=True
)
fig_revenue.update_layout(
    title_font_size=20,
    title_x=0.5,
    height=500,
        plot_bgcolor='#3a4f63',
        paper_bgcolor='#3a4f63',
    font=dict(color='white', family='Roboto')
)
fig_revenue.write_html("top10_Revenue.html")

# --- Quantity Chart ---
chart_data_quantity = item_metrics.sort_values(by='Quantity', ascending=False).head(10)
fig_quantity = px.bar(
    chart_data_quantity,
    x='item_name',
    y='Quantity',
    title='Top Items by Quantity',
    color='item_name',
    text_auto=True
)
fig_quantity.update_layout(
    title_font_size=20,
    title_x=0.5,
    height=500,
        plot_bgcolor='#3a4f63',
        paper_bgcolor='#3a4f63',
    font=dict(color='white', family='Roboto')
)
fig_quantity.write_html("top10_Quantity.html")

# --- Frequency Chart ---
chart_data_frequency = item_metrics.sort_values(by='Frequency', ascending=False).head(10)
fig_frequency = px.bar(
    chart_data_frequency,
    x='item_name',
    y='Frequency',
    title='Top Items by Frequency',
    color='item_name',
    text_auto=True
)
fig_frequency.update_layout(
    title_font_size=20,
    title_x=0.5,
    height=500,
        plot_bgcolor='#3a4f63',
        paper_bgcolor='#3a4f63',
    font=dict(color='white', family='Roboto')
)
fig_frequency.write_html("top10_Frequency.html")

# --- AvgPrice Chart ---
chart_data_avgprice = item_metrics.sort_values(by='AvgPrice', ascending=False).head(10)
fig_avgprice = px.bar(
    chart_data_avgprice,
    x='item_name',
    y='AvgPrice',
    title='Top Items by Average Price',
    color='item_name',
    text_auto=True
)
fig_avgprice.update_layout(
    title_font_size=20,
    title_x=0.5,
    height=500,
        plot_bgcolor='#3a4f63',
        paper_bgcolor='#3a4f63',
    font=dict(color='white', family='Roboto')
)
fig_avgprice.write_html("top10_AvgPrice.html")

In [7]:
import pandas as pd
import plotly.express as px

# Load and preprocess data
df = pd.read_excel(r'C:\Users\TUF\Desktop\Franz FIles\Balaji-Fast-Food-Sales-Cleaned.xlsx')

df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df['transaction_amount'] = pd.to_numeric(df['transaction_amount'], errors='coerce')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['transaction_amount', 'date'])

order_behavior = df.groupby('order_id').agg(
    TotalRevenue=('transaction_amount', 'sum'),
    TotalQuantity=('quantity', 'sum'),
    UniqueItems=('item_name', 'nunique'),
    AvgItemPrice=('item_price', 'mean'),
    OrderDate=('date', 'min')
).reset_index()

order_behavior['Segment'] = pd.cut(
    order_behavior['TotalRevenue'],
    bins=[0, 500, 1500, 3000, float('inf')],
    labels=['Low', 'Medium', 'High', 'Very High']
)

layout_style = dict(
    title_x=0.5,
    title_font=dict(color='white'),
    font=dict(color='white'),
    legend=dict(font=dict(color='white')),
    xaxis=dict(title_font=dict(color='white'), tickfont=dict(color='white')),
    yaxis=dict(title_font=dict(color='white'), tickfont=dict(color='white')),
    plot_bgcolor='#3a4f63',
    paper_bgcolor='#3a4f63'
)

# --- Scatter Plot ---
fig1 = px.scatter(
    order_behavior,
    x='TotalQuantity',
    y='TotalRevenue',
    color='Segment',
    size='UniqueItems',
    title='🔵 Revenue vs Quantity',
    template='plotly_white',
    hover_data=['order_id', 'AvgItemPrice']
)
fig1.update_traces(marker=dict(opacity=0.7, line=dict(width=1, color='gray')))
fig1.update_layout(**layout_style)
fig1.write_html("scatter_plot.html")

# --- Correlation Matrix ---
corr = order_behavior[['TotalRevenue', 'TotalQuantity', 'UniqueItems', 'AvgItemPrice']].corr()
fig2 = px.imshow(
    corr,
    text_auto=True,
    title="📊 Correlation Matrix",
    color_continuous_scale='Blues',
    template='plotly_white'
)
fig2.update_layout(**layout_style)
fig2.write_html("correlation_matrix.html")

# --- Outliers by Segment ---
fig3 = px.box(
    order_behavior,
    x='Segment',
    y='TotalRevenue',
    color='Segment',
    points='all',
    title='🚨 Revenue Outliers by Segment',
    template='plotly_white'
)
fig3.update_layout(**layout_style)
fig3.write_html("outliers_by_segment.html")

# --- Segment Bar Chart ---
segment_counts = order_behavior['Segment'].value_counts().sort_index()
fig4 = px.bar(
    x=segment_counts.index,
    y=segment_counts.values,
    labels={'x': 'Segment', 'y': 'Number of Orders'},
    title='📦 Customer Segments (Simple Bar)',
    color=segment_counts.index,
    template='plotly_white'
)
fig4.update_layout(**layout_style)
fig4.write_html("segments_bar_chart.html")

# --- Line Trend Over Time ---
revenue_trend = order_behavior.groupby('OrderDate').agg(
    AvgRevenue=('TotalRevenue', 'mean')
).reset_index()
fig5 = px.line(
    revenue_trend,
    x='OrderDate',
    y='AvgRevenue',
    title='📈 Avg Revenue Over Time',
    template='plotly_white'
)
fig5.update_traces(line=dict(color='royalblue', width=3))
fig5.update_layout(**layout_style)
fig5.write_html("avg_revenue_trend.html")
fig5.show()


In [8]:
import pandas as pd
import plotly.express as px
from datetime import datetime

# Load data (same Excel file)
df = pd.read_excel(r'C:\Users\TUF\Desktop\Franz FIles\Balaji-Fast-Food-Sales-Cleaned.xlsx')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.dropna(subset=['date', 'transaction_amount'], inplace=True)
df.rename(columns={
    'transaction_amount': 'Revenue',
    'item_type': 'Category'
}, inplace=True)

# Create month and quarter columns
df['Month'] = df['date'].dt.to_period('M').dt.to_timestamp()
df['Quarter'] = df['date'].dt.to_period('Q').dt.to_timestamp()

# Color palette for dark background
custom_colors = ['#ffd166', '#06d6a0', '#ef476f', '#118ab2', '#ffffff']

# GROWTH RATE
monthly = df.groupby('Month')['Revenue'].sum().reset_index()
monthly['Growth'] = monthly['Revenue'].pct_change() * 100
fig = px.bar(
    monthly, x='Month', y='Growth',
    title='Month-over-Month Growth (%)',
    color_discrete_sequence=custom_colors
)
fig.update_layout(
    template='plotly_white',
    height=500,
    xaxis_title='Date',
    yaxis_title='Growth %',
    title_font=dict(size=22, family='Arial', color=custom_colors[2]),
    font=dict(family='Arial', color='#fff'),
    plot_bgcolor='#3a4f63',
    paper_bgcolor='#3a4f63'
)
fig.write_html("monthly_sales_chart_growth_DUP.html")
fig.show()

In [9]:
# --- Duplicate: Line Trend Over Time ---
revenue_trend = order_behavior.groupby('OrderDate').agg(
    AvgRevenue=('TotalRevenue', 'mean')
).reset_index()

fig_dup = px.line(
    revenue_trend,
    x='OrderDate',
    y='AvgRevenue',
    title='📈 Avg Revenue Over Time',
    template='plotly_white'
)
fig_dup.update_traces(line=dict(color='mediumseagreen', width=3))
fig_dup.update_layout(**layout_style)
fig_dup.write_html("avg_revenue_trend_DUP.html")
fig_dup.show()

In [12]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import plotly.graph_objects as go

df = pd.read_excel(r'C:\Users\TUF\Desktop\Franz FIles\Balaji-Fast-Food-Sales-Cleaned.xlsx')
df['date'] = pd.to_datetime(df['date'])
df['Month'] = df['date'].dt.to_period('M').dt.to_timestamp()

# Output folder
output_dir = 'forecast_htmls'
os.makedirs(output_dir, exist_ok=True)

items = df['item_name'].unique()
months_range = [3, 6, 9, 12]  # You can add more if needed

for item in items:
    filtered = df[df['item_name'] == item]
    monthly = filtered.groupby('Month')['transaction_amount'].sum().reset_index()

    if len(monthly) < 3:
        continue

    monthly['TimeIndex'] = range(len(monthly))
    model = LinearRegression().fit(monthly[['TimeIndex']], monthly['transaction_amount'])

    for months in months_range:
        future_idx = np.arange(len(monthly), len(monthly) + months)
        future_dates = pd.date_range(start=monthly['Month'].iloc[-1] + pd.offsets.MonthBegin(1), periods=months, freq='MS')
        future_preds = model.predict(future_idx.reshape(-1, 1))

        forecast_df = pd.DataFrame({'Month': future_dates, 'transaction_amount': future_preds, 'Type': 'Forecast'})
        actual_df = monthly[['Month', 'transaction_amount']].copy()
        actual_df['Type'] = 'Actual'

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=actual_df['Month'], y=actual_df['transaction_amount'], name='Actual',
                                 mode='lines+markers', line=dict(color='deepskyblue', width=3)))
        fig.add_trace(go.Scatter(x=forecast_df['Month'], y=forecast_df['transaction_amount'], name='Forecast',
                                 mode='lines+markers', line=dict(color='gold', dash='dash', width=3)))

        fig.update_layout(
            title=f'Forecast: {item} | Next {months} Months',
            xaxis_title='Month',
            yaxis_title='Transaction Amount',
            plot_bgcolor='#3a4f63',
            paper_bgcolor='#3a4f63',
            font=dict(color='white'),
            height=500
        )

        file_name = f"{item.replace(' ', '_').lower()}_{months}months.html"
        fig.write_html(f"{output_dir}/{file_name}")

c:\Users\TUF\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\TUF\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\TUF\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\TUF\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\TUF\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\TUF\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\T